In [1]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

In [2]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [3]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [4]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

In [5]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_NO_Privat_ohneApple_Langzeitkunde.csv'

df_NO_werte = pd.read_csv(os.path.join(path, filename))

In [6]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [7]:
## Alle DataFrame Werte von Float zu int
df_trans_werte = df_trans_werte.astype(int)
df_LY = df_LY.astype(int)
df_HY = df_HY.astype(int)
df_NO_werte = df_NO_werte.astype(int)
df_LO = df_LO.astype(int)

# Transwerte-1 definiert für spätere Berecnungen in der Copula
#df_trans_werte_minusEins = df_trans_werte.loc[:, df_NO_werte.columns != 'iid'] -1
#df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

In [8]:
len(df_trans_werte)

1462

In [9]:
len(df_LY)

1462

In [10]:
len(df_HY)

697

In [11]:
len(df_NO_werte)

647

In [12]:
len(df_LO)

647

### Sicherstellen dass alle df dieselben Kunden verwenden

In [13]:
# Entfernen der Zeilen aus df_HY, die nicht in df_trans_werte und df_LO enthalten sind
df_HY = df_HY[df_HY['iid'].isin(df_trans_werte['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LO['iid'])]

# Entfernen der Zeilen aus df_trans_werte, die nicht in df_HY, df_NO_werte und df_LY enthalten sind
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_NO_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_LY['iid'])]

# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LO = df_LO[df_LO['iid'].isin(df_HY['iid'])]

# Entfernen der Zeilen aus df_NO_werte, die nicht in df_trans_werte enthalten sind
df_NO_werte = df_NO_werte[df_NO_werte['iid'].isin(df_trans_werte['iid'])]

# Entfernen der Zeilen aus df_LY, die nicht in df_trans_werte enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_trans_werte['iid'])]

### Alle DataFrames Indizes resetten, damit man sie gemeinsam identifiziere kann
df_HY = df_HY.reset_index(drop=True)
df_trans_werte = df_trans_werte.reset_index(drop=True)
df_LO = df_LO.reset_index(drop=True)
df_NO_werte = df_NO_werte.reset_index(drop=True)
df_LY = df_LY.reset_index(drop=True)

# NO-1 definiert für spätere Berechnung inder Copula
df_NO_minusEins = df_NO_werte.copy()
df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] = df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] -1
df_NO_minusEins = df_NO_minusEins.astype(int)
# transwerte -1 definiert für spätere Berechnung inder Copula
df_trans_werte_minusEins = df_trans_werte.copy()
df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] = df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] -1
df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

In [14]:
# Siherstellen dass alle dataframes die Kunden in derselben Reihenfolge auflisten
df_NO_minusEins = df_NO_minusEins.sort_values('iid')
df_HY = df_HY.sort_values('iid')
df_trans_werte = df_trans_werte.sort_values('iid')
df_LO = df_LO.sort_values('iid')
df_NO_werte = df_NO_werte.sort_values('iid')
df_LY = df_LY.sort_values('iid')
df_trans_werte_minusEins = df_trans_werte_minusEins.sort_values('iid')

#### Ausreißer bei den Käufen ermitteln

In [15]:
# Darstellun dergesamte Käufe der Kunden
arr = np.sum(df_trans_werte[df_trans_werte.columns[df_trans_werte.columns != 'iid']], axis=1)

# Sortiere das Array nach Größe
sorted_arr = np.sort(arr)

print(sorted_arr)

[ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5

In [16]:
# Finde die Indizes der größten Einträge
indices_ausreisser_käufe = np.argsort(arr)[-10:]

# Gebe die 10 Zeilen mit den größten Einträgen und deren Positionen aus
for index in indices_ausreisser_käufe:
    print("Gesamtzahl Käufe:", arr[index], "Position:", index)

Gesamtzahl Käufe: 31 Position: 415
Gesamtzahl Käufe: 32 Position: 451
Gesamtzahl Käufe: 32 Position: 531
Gesamtzahl Käufe: 33 Position: 364
Gesamtzahl Käufe: 34 Position: 351
Gesamtzahl Käufe: 34 Position: 348
Gesamtzahl Käufe: 37 Position: 501
Gesamtzahl Käufe: 43 Position: 408
Gesamtzahl Käufe: 44 Position: 285
Gesamtzahl Käufe: 92 Position: 422


In [17]:
for column in df_trans_werte.columns[df_trans_werte.columns != 'iid']:
    largest_values = df_trans_werte[column].nlargest(3)
    print(f"Die drei größten Werte in der Spalte '{column}' sind:")
    print(largest_values)
    print("Die zugehörigen Indizes sind:")
    print(largest_values.index)
    print()

Die drei größten Werte in der Spalte '1' sind:
351    7
408    7
376    5
Name: 1, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([351, 408, 376], dtype='int64')

Die drei größten Werte in der Spalte '2' sind:
251    10
422     9
351     8
Name: 2, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([251, 422, 351], dtype='int64')

Die drei größten Werte in der Spalte '3' sind:
364    9
422    9
408    4
Name: 3, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([364, 422, 408], dtype='int64')

Die drei größten Werte in der Spalte '4' sind:
422    7
531    7
32     5
Name: 4, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([422, 531, 32], dtype='int64')

Die drei größten Werte in der Spalte '5' sind:
415    4
422    4
597    4
Name: 5, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([415, 422, 597], dtype='int64')

Die drei größten Werte in der Spalte '6' sind:
422    15
154     5
285     5
Name: 6, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([422, 15

In [18]:
# iids mit Käufen >= 8 in einem Monat, die nicht bereits in indices_ausreisser_käufe sind:
indices_ausreisser_käufe_2 = [251, 220]

In [19]:
indices_ausreisser_käufe = np.concatenate([indices_ausreisser_käufe, indices_ausreisser_käufe_2])

#### Ausreißer bei den Öffungen ermitteln

In [20]:
for column in df_NO_werte.columns[df_NO_werte.columns != 'iid']:
    largest_values = df_NO_werte[column].nlargest(3)
    print(f"Die drei größten Werte in der Spalte '{column}' sind:")
    print(largest_values)
    print("Die zugehörigen Indizes sind:")
    print(largest_values.index)
    print()

Die drei größten Werte in der Spalte '1' sind:
140    3
145    3
152    3
Name: 1, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([140, 145, 152], dtype='int64')

Die drei größten Werte in der Spalte '2' sind:
37     3
69     3
166    3
Name: 2, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([37, 69, 166], dtype='int64')

Die drei größten Werte in der Spalte '3' sind:
443    5
642    3
1      2
Name: 3, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([443, 642, 1], dtype='int64')

Die drei größten Werte in der Spalte '4' sind:
387    8
408    5
533    4
Name: 4, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([387, 408, 533], dtype='int64')

Die drei größten Werte in der Spalte '5' sind:
35    3
75    3
87    3
Name: 5, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([35, 75, 87], dtype='int64')

Die drei größten Werte in der Spalte '6' sind:
8     4
9     4
43    4
Name: 6, dtype: int32
Die zugehörigen Indizes sind:
Int64Index([8, 9, 43], dtype='int64')

In [21]:
indices_ausreisser_öffnungen = [443, 387, 408]

## Ausreißer bei den Kunden löschen

In [22]:
# Ausreißer Käufe = indices_ausreisser_käufe
# Ausreißer Öffungen = indices_ausreisser_öffnungen
ausreisser_pos = np.concatenate([indices_ausreisser_öffnungen, indices_ausreisser_käufe])
ausreisser_pos

array([443, 387, 408, 415, 451, 531, 364, 351, 348, 501, 408, 285, 422,
       251, 220], dtype=int64)

In [23]:
# Zeilen löschen
df_NO_minusEins = df_NO_minusEins.drop(ausreisser_pos).reset_index(drop = True)
df_HY = df_HY.drop(ausreisser_pos).reset_index(drop = True)
df_trans_werte = df_trans_werte.drop(ausreisser_pos).reset_index(drop = True)
df_LO = df_LO.drop(ausreisser_pos).reset_index(drop = True)
df_NO_werte = df_NO_werte.drop(ausreisser_pos).reset_index(drop = True)
df_LY = df_LY.drop(ausreisser_pos).reset_index(drop = True)
df_trans_werte_minusEins = df_trans_werte_minusEins.drop(ausreisser_pos).reset_index(drop = True)

## Spezifikation des Modells

In [24]:
anzahl_kunden = len(df_trans_werte["iid"])
anzahl_kunden

633

In [25]:
anzahl_monate = len(df_trans_werte.columns[df_trans_werte.columns != 'iid'])
anzahl_monate

15

In [26]:
anzahl_states = 3

##### Definition der Transitionmatrix

In [73]:
# Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat):
    
    state_t_minusEins = state_t_minusEins -1
    state = state -1
    str_monat = str(monat)
    
    # Definition der X_t_minusEins
    # verwendete Parameter
    EM_monat_minusEins = df_HY[str_monat]
    Y_monat_minusEins = df_trans_werte[str_monat]
    O_monat_minusEins = df_NO_werte[str_monat]
    # Definition
    X_t_minusEins = [O_monat_minusEins.apply(lambda x: int(x > 0)),\
                     Y_monat_minusEins.apply(lambda x: int(x > 0)),\
                     EM_monat_minusEins, np.square(EM_monat_minusEins)]
    
    X_t_minusEins = np.array(X_t_minusEins)
    array_v = alpha[anzahl_states*state_t_minusEins + state] +\
                    gamma[anzahl_states*state_t_minusEins + state] * X_t_minusEins[0] +\
                    gamma[(np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[1] +\
                    gamma[(2* np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[2] +\
                    gamma[(3* np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[3]

    return array_v

# Definition der Transitionmatrix
def probability_Q_state_monat(alpha, gamma):
    
    ### Erstellung leerer anzahl_states x anzahl_states arrays für jeden Kunden und jeden Monat
    Q = np.zeros((anzahl_kunden, anzahl_monate - 1, anzahl_states, anzahl_states))
                
    for monat in range(2, anzahl_monate + 1):
        
        for state_t_minusEins in range(1, anzahl_states +1):
            
            # Erste Spalte besteht aus Nullen, da eine Trasition zurück in state 1 nicht möglich ist
            for state in range(2, anzahl_states +1):
                
                q_array_iid_monat_state = Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat)

                # Speicherung der Werte in den leeren arrays
                for iid in range(0, anzahl_kunden):
                    
                    #Neue Definition ohne die Verwedung von Namen
                    Q[iid, monat-2, state_t_minusEins-1, state-1] = q_array_iid_monat_state[iid]
                    
    
    # Wenn einmal in state 2, kann nicht mehr zurück in state 1
    Q[:,:,:,0] = 0.0
    
    #Anpassung: 
    #denominator = 1 + np.sum(np.exp(Q[...,1:]), axis=3, keepdims=True)
    #Q = np.exp(Q)/denominator
    
    denominator = np.sum(np.exp(Q[...,1:]), axis=3, keepdims=True)
    Q = np.exp(Q)/denominator
    
    return Q

#### Tests

##### Definition der Verteilungsfunktion F_1 von CEOM aals anzahl_kunden x 1 Array für jeden state und Monat

In [28]:
# Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def lambda_O_state_monat(alpha_o, beta_o, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    LO_monat = df_LO[str_monat]
    
    # Erhalten der Werte für die Verarbeitung in der Formel
    array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))

    return array_lambda

# Array der Verteilungsfunktion der Öffungen O für Monat und state
def F_O_state_monat(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_werte[str_monat]
    
    lambda_O_state_monat_berechnung = lambda_O_state_monat(alpha_o, beta_o, state, monat)
    
    array_F_O_berechnung = poisson.cdf(o_monat, lambda_O_state_monat_berechnung)
         
    return array_F_O_berechnung

# Array der Verteilungsfunktion der Öffungen O -1 für Monat und state
def F_O_state_monat_minusEins(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_minusEins[str_monat]
    
    lambda_O_state_monat_berechnung = lambda_O_state_monat(alpha_o, beta_o, state, monat)
    
    array_F_O_berechnung = poisson.cdf(o_monat, lambda_O_state_monat_berechnung)

    return array_F_O_berechnung

#### Tests

##### Definition der Verteilungsfunktio F_2 von CPM als anzahl_kunden x 1 Array für jeden state und Monat

In [57]:
# Definition der lambdaY als anzahl_iids x 1 array für monat und state
def p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    LY_monat = df_LY[str_monat]
    
    array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat) +\
                      + delta_1[state] * np.log(LY_monat))
    array_p = array_exp / (1+ array_exp)
    
    return array_p

# Array der Verteilungsfunktion der Käufe Y für Monat und state
def F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # y_monat ist das array mit der Anzahl an Käufen in monat str_monat
    y_monat = df_trans_werte[str_monat]
    LY_monat = df_LY[str_monat]
    # p ist das 1 x anzahl_iids array mit p für jeden kunden mit gegegebenem monat und state
    array_p = p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
    # cdf berechnen
    cdf = nbinom.cdf(y_monat, 0.779, array_p)
    
    return cdf

# Array der Verteilungsfunktion der Käufe Y-1 für Monat und state
def F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # y_monat ist das array mit der Anzahl an Käufen in monat str_monat
    y_monat = df_trans_werte[str_monat] -1
    LY_monat = df_LY[str_monat]
    # p ist das 1 x anzahl_iids array mit p für jeden kunden mit gegegebenem monat und state
    array_p = p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
    # cdf berechnen
    cdf = nbinom.cdf(y_monat, 0.779, array_p)
    
    return cdf

#### Tests

##### Definition der Frank-copula

In [30]:
# Frank Copula, theta fehlt
def Frank_copula(u_1, u_2, theta):
    numerator = (np.exp(-theta[0]*u_1) - 1) * (np.exp(-theta[0]*u_2) - 1)
    denominator = np.exp(-theta[0]) - 1
    fraction = numerator / denominator
    return (-1 / theta[0]) * np.log(1 + fraction)

##### Definition der Likelihoodfunktion

In [31]:
# alpha_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2 sind state dependent
# beta_o, r sind state independent
def likelihood_function(params):
    
    alpha_size = anzahl_states**2
    gamma_size = anzahl_states**2 * 4

    alpha = params[:alpha_size]
    gamma = params[alpha_size:alpha_size+gamma_size]
    alpha_o = params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
    beta_o = params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
    delta_0 = params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
    delta_1 = params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
    alpha_p = params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
    beta_p_1 = params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
    beta_p_2 = params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
    theta = params[alpha_size+gamma_size+anzahl_states*6+1:]


    #alpha, gamma, alpha_o, beta_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, r, theta = params
    ### Erstellung einer Liste H die alle H_it enthalten
    H = np.zeros((anzahl_kunden, anzahl_monate, anzahl_states))
                
                
    for monat in range(1,anzahl_monate+1):
        for state in range(1, anzahl_states +1):
            
            # Berechnung von arrays für die Verteilungsfunktionen
            F_O_state_monat_berechnung = F_O_state_monat(alpha_o, beta_o, state, monat)
            
            F_O_state_monat_berechnung_minusEins = F_O_state_monat_minusEins(alpha_o, beta_o, state, monat)
            
            F_Y_state_monat_berechnung = F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
            
            F_Y_state_monat_berechnung_minusEins =\
                    F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
            
            # Berechnung der gemeinsamen Verteilungen
            H_state_monat_berechnung =\
                            Frank_copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung, theta) -\
                            Frank_copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung, theta) -\
                            Frank_copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung_minusEins, theta) +\
                            Frank_copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung_minusEins, theta)
            
            # Speicherung der Werte in den leeren arrays
            H[:, monat-1, state-1] = H_state_monat_berechnung

    # Startverteilung ist (1, 0,..., 0)
    startverteilung = np.zeros((1, anzahl_states))
    startverteilung[0, 0] = 1
    
    # Transitionmatrix Q, liste von 3x3 Arrays, wobei für Monate 2, ..., 15 die Q_it angegeben werden und die Q_it fü die
    # nächsten Kunden dahinter angehängt werden
    Q = probability_Q_state_monat(alpha, gamma)
    
    eins_vektor = np.ones((anzahl_states, 1))
    
    likelihood = np.array([])
    
    for iid in range(anzahl_kunden):
        result = startverteilung
        result = np.matmul(result, np.diag(H[iid, 0]))
        H_iid = np.eye(anzahl_states)
        for monat in range(2, anzahl_monate+1):
                H_iid_monat = np.matmul(np.diag(H[iid, monat-1]), Q[iid, monat-2])
                H_iid = np.matmul(H_iid, H_iid_monat)
        result = np.matmul(result, H_iid)
        result = np.matmul(result, eins_vektor)
        likelihood = np.concatenate((likelihood, result[0]))
        
    return abs(np.sum(np.log(likelihood)))

#### Tests

In [32]:
values_quelle_alpha_o = [0]
alpha_o = values_quelle_alpha_o * anzahl_states
alpha_o

[0, 0, 0]

# Berechnung der Likelihoodfunktion und Schätzung der Parameter

### Dispersion Parameter r für die ZINBD Verteilun wurde vorhergeschätzt

In [33]:
r = [1.29]

### Definition der Startwerte

In [34]:
# Parameter für Transitionmatrix
values_quelle_alpha = [0, -2.647, -1.432, 0, 2.829, -3.815, 0, -2.398, 0.61]
alpha = np.array(values_quelle_alpha)
gamma = np.zeros((anzahl_states, anzahl_states, 4)).flatten()

# Parameter für CEOM
# values_quelle_alpha_o wurde aus einer vorherigen Optimierung übernommen
values_quelle_alpha_o = [0]
alpha_o = values_quelle_alpha_o * anzahl_states
# values_quelle_beta_o wurdevorher nicht geschätzt
values_quelle_beta_o = [0]
beta_o = values_quelle_beta_o

# Parameter für CPM
# values_quelle_alpha_o wurde aus einer vorherigen Optimierung übernommen
values_quelle_delta_0 = [np.log(1.29)]
delta_0 = values_quelle_delta_0 * anzahl_states
# values_quelle_delta_1 wurde nicht aus einer vorherigen Optimierung übernommen
values_quelle_delta_1 = [0]
delta_1 = values_quelle_delta_1 * anzahl_states
# values_quelle_alpha_p wurde aus einer vorherigen Optimierung übernommen
values_quelle_alpha_p = [np.log(0.779)]
alpha_p = values_quelle_alpha_p * anzahl_states
# beta_p_1 wurde nicht aus einer vorherigen Optimierung übernommen
values_quelle_beta_p_1 = [0]
beta_p_1 = values_quelle_beta_p_1 * anzahl_states
# beta_p_2 wurde nicht aus einer vorherigen Optimierung übernommen
values_quelle_beta_p_2 = [0]
beta_p_2 = values_quelle_beta_p_2 * anzahl_states

# Parameter für Frank_copula
theta = np.array([0.324])

In [35]:
# Startwerte
x0 = np.concatenate((
    alpha,
    gamma,
    alpha_o,
    beta_o,
    delta_0,
    delta_1,
    alpha_p,
    beta_p_1,
    beta_p_2,
    theta))

### Sonderfall

In [36]:
#x0 = result.x

### Definition der Bounds und Constraints

In [62]:
# Definiere bounds
bounds = [(-5, 5)] * (len(x0) - 1) + [(0.1, None)]

In [63]:
#Für die Definition der constraint relevant
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4
# alpha_o = params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
constraint_matrix = np.zeros((2, len(x0)))
# Positionen für alpha_o
alpha_o_1_pos = alpha_size + gamma_size
alpha_o_2_pos = alpha_size + gamma_size + 1
alpha_o_3_pos = alpha_size + gamma_size + 2
# -1 und 1 an den entsprechenden Positionen setzen
constraint_matrix[0][alpha_o_1_pos] = -1
constraint_matrix[0][alpha_o_2_pos] = 1
constraint_matrix[1][alpha_o_2_pos] = -1
constraint_matrix[1][alpha_o_3_pos] = 1

In [64]:
constraint_matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [65]:
linear_constraint = LinearConstraint(constraint_matrix, [0, 0], [np.inf, np.inf])

### Berechnung per minimize Funktion

In [84]:
# Rufen Sie minimize() auf und übergeben Sie die zusätzlichen Parameter mit args=
options = {'maxiter': 100} 
result = minimize(likelihood_function, x0, options=options, bounds = bounds, method = "Nelder-Mead")
                  #args=(alpha, gamma, alpha_o, beta_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, r, theta), bounds=bounds)

# Gib die geschätzten Parameter aus
print(result.x)

[ 0.         -2.647      -1.5036      0.          2.829      -3.815
  0.         -2.398       0.61        0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.25464222  0.25464222  0.25464222  0.          0.
  0.         -0.24974423 -0.24974423 -0.24974423  0.          0.
  0.          0.          0.          0.          0.324     ]


In [85]:
# Berechne die Likelihood für die optimalen Parameter
likelihood = likelihood_function(result.x)
print("Likelihood für optimale Parameter:")
print(likelihood)

Likelihood für optimale Parameter:
15330.779419126433


In [86]:
print(result.success)

False


In [87]:
result

 final_simplex: (array([[ 0.00000000e+00, -2.64700000e+00, -1.50360000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  3.24000000e-01],
       [-1.63149378e-05, -2.69088585e+00, -1.45574180e+00, ...,
         2.53984603e-05,  3.19626583e-05,  3.21885877e-01],
       [ 0.00000000e+00, -2.77935000e+00, -1.43200000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  3.24000000e-01],
       ...,
       [ 7.98511943e-06, -2.65949724e+00, -1.43876088e+00, ...,
         2.36064212e-05,  2.36064212e-05,  3.22777877e-01],
       [-2.34790556e-04, -2.65912419e+00, -1.43855906e+00, ...,
         2.29017519e-05,  2.29017519e-05,  3.22814358e-01],
       [ 2.22181175e-05, -2.65876227e+00, -1.43836327e+00, ...,
         2.22181175e-05,  2.22181175e-05,  3.22849750e-01]]), array([15330.77941913, 15519.44877241, 15520.85705873, 15522.74971915,
       15526.06188966, 15529.38166434, 15532.70563013, 15536.03057138,
       15539.35346109, 15542.67221262, 15545.98305067, 15550.17399902,
       15553

# Ergebisse:

In [ ]:
# Ausgangssituation
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda / 1 + array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 17812")

In [ ]:
# Test ob array_lambda = array_lambda / 1 + array_lambda bessere oder schlechtere Ergebnisse liefert 
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 17812")

In [ ]:
# Test kein quadratischer Einfluss des EM auf Käufe
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat)")
print("Likelihood für optimale Parameter: 17812")

In [ ]:
# Test, ob keine "keine constraints" zu besseren Ergebnissen führt
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 17547")

In [ ]:
#Ob nach Anpassungen die schönere Schreibweise auch für die ZINBD Definition funktioeiert und dieselben Ergebnisse liefert
print("likelihood mit ZINBD: 14451")

In [ ]:
# Test ob für Standardspezifikation
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 14451")
print("Parameter:")

In [ ]:
optim_param = [ 0.00000000e+00, -8.42013774e+01,  1.69986125e+01,  0.00000000e+00,
       -1.25200153e+01, -1.53060979e+01,  0.00000000e+00, -4.01802666e+00,
        3.14514828e+01,  0.00000000e+00, -3.40875207e+01, -2.82844507e-01,
        0.00000000e+00, -1.16977610e+01, -4.21223535e+01,  0.00000000e+00,
       -3.23512569e+01,  5.05520432e+00,  0.00000000e+00, -1.36423722e+00,
       -1.19458664e+00,  0.00000000e+00,  1.05242438e+00, -1.51554184e+01,
        0.00000000e+00,  1.02752609e+00,  8.49398978e-01,  0.00000000e+00,
        7.18296331e+01,  4.41698548e-01,  0.00000000e+00,  1.58841432e+01,
        1.58119714e+01,  0.00000000e+00,  1.02333573e+01, -1.84782829e+01,
        0.00000000e+00, -1.06581824e+01, -4.26447692e-01,  0.00000000e+00,
        2.48374764e+01,  2.47507462e+01,  0.00000000e+00, -1.02161982e+00,
        4.57256224e+00,  2.68038255e-02, -9.78697275e-01,  2.34488631e-01,
       -2.46871753e-01,  1.80545838e+01,  2.12346810e+01,  1.49348847e+01,
        0.00000000e+00,  1.72348151e-01, -3.64387979e-01, -5.53773619e-01,
       -2.76890669e+00, -1.66599130e+00,  4.94843823e-01,  8.64751297e-01,
       -6.48676929e-02, -2.92112867e-01, -1.55380568e-01,  5.61001901e-02,
        1.47689589e+00]

In [ ]:
# Test ob np.log(0.63)*EM_monat besser als np.log(0.63*EM_monat)läuft und evtl terminiert
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18522")

In [ ]:
optim_param = [ 0.00000000e+00, -8.42324287e+01,  1.69437089e+01,  0.00000000e+00,
       -1.51114259e+01, -1.27787560e+01,  0.00000000e+00, -4.66039409e+00,
        3.23378060e+01,  0.00000000e+00, -3.42059247e+01, -3.03032101e-01,
        0.00000000e+00, -1.16969644e+01, -4.21223535e+01,  0.00000000e+00,
       -3.23512569e+01,  5.10330353e+00,  0.00000000e+00, -1.34546356e+00,
       -1.45060842e+00,  0.00000000e+00,  1.05242438e+00, -1.51554184e+01,
        0.00000000e+00,  6.50267936e-01,  1.11128688e+00,  0.00000000e+00,
        7.18149941e+01,  5.07482709e-01,  0.00000000e+00,  1.65202863e+01,
        1.50351002e+01,  0.00000000e+00,  9.40601150e+00, -1.66324994e+01,
        0.00000000e+00, -1.05955017e+01, -1.72078495e-01,  0.00000000e+00,
        2.62869932e+01,  2.28266666e+01,  0.00000000e+00, -1.69422693e+00,
        7.29303793e+00,  4.87220599e-01,  7.01288616e-01,  1.24299189e+00,
       -1.36993482e+00,  1.80247785e+01,  2.12346810e+01,  1.55971024e+01,
        0.00000000e+00,  1.72348151e-01, -5.49982853e-01, -2.11311398e-01,
       -3.31995657e+00, -1.50224012e+00,  2.04106841e-01,  1.71997448e+00,
       -8.41039175e-02, -2.74031043e-01, -4.31434986e-01,  1.32125605e-02,
        1.07630364e+00]

In [ ]:
# Test ob np.log(0.63)*EM_monat besser als np.log(0.63*EM_monat)läuft und evtl terminiert
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18522")
print("result.success: False")

In [ ]:
# Test ob Q[:,:,:,0] ungleich Null zu besseren Ergebnissen führt
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18716")

In [ ]:
optim_param_positiveQ_state1_Nelder = [-7.44841087e-01, -8.85139802e+02, -2.40481934e+02, -2.05213086e+00,
       -5.60706087e+02, -5.35441302e+02, -9.63275246e-03, -4.27150076e+02,
       -1.92413334e+01,  1.87679188e+00,  1.58999244e-01, -9.03253128e-02,
       -3.28630690e-01, -2.43215232e-01, -1.37190579e-01,  3.42037204e-01,
        2.51357509e-01,  2.06682854e-01, -4.73934832e-01, -2.71680222e-01,
        1.60689390e-01, -1.72473497e-01, -2.60376329e-01,  2.51404015e-01,
       -1.00917729e-02, -7.52990475e-01,  2.61244903e-02,  7.95823527e-01,
       -5.31101017e-02, -2.62763614e-01,  8.73203616e-01, -7.32946413e-01,
       -3.99863957e-01,  5.41473133e-01,  1.89988509e+00, -7.65683186e-01,
        5.79608497e-01,  1.28236010e-01,  9.41770971e-01, -8.84228191e-01,
       -7.05129175e-01, -2.51729254e+00, -1.31076750e+00,  1.76696263e-01,
        3.34715279e-01,  1.11964744e+00,  3.44350612e-01,  3.11862570e+00,
       -1.35299604e+00,  9.91086034e+01,  5.99497355e+00, -4.56900519e+01,
       -4.73360515e-01, -7.11949781e-01,  6.54292220e-02, -1.26160660e+00,
        7.29158556e+01, -3.38290584e+01, -2.22817443e-01,  8.27814530e-02,
       -4.07211009e-01,  5.30692948e-02, -8.46733418e-01, -5.21951998e-01,
        4.65391969e-01]

In [ ]:
# Test ob Q[:,:,:,0] ungleich Null, Nelder-Mead, vorherige Datenbereinigung, 30k max iter zu besseren Ergebnissen führt
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18116")

In [ ]:
optim_param_positiveQ_state1_Nelder_30k = [ 4.17928553e-01, -4.22285316e+02, -1.98242290e+00, -8.79879518e-01,
       -1.94045445e+03,  6.99046727e+02,  9.13766559e-01, -5.31026400e+02,
        4.31211962e+01, -2.14401838e+00, -1.67642789e+00,  6.08254147e-01,
       -5.04744507e-03,  1.87801747e-02,  6.87473148e-01, -3.36485684e-01,
        1.29745132e+00,  8.72972789e-01, -3.52212973e-01, -4.02096620e-01,
        4.45619697e-01, -6.69209684e-01,  1.44688445e+00,  2.53356872e-01,
        1.80355515e-02, -1.11375261e+00, -4.15990883e-01, -4.54538186e-01,
       -8.26245723e-01, -4.51785453e-01,  5.19280797e-01, -1.28267812e+00,
        1.47131184e+00, -5.86812259e-01,  6.62504829e-01,  1.97691597e+00,
        4.27950006e-01, -3.43138559e+00,  1.03359822e+00,  7.07554502e-02,
        7.10101998e-01,  5.84550086e-01,  1.33027166e+00,  2.22558553e-01,
        1.00029849e+00,  1.36650013e-01, -1.19263433e+00,  1.21048688e+00,
       -1.13572566e+00,  9.56897194e+01,  1.37166797e+01,  2.70925515e+01,
        1.93039105e+00, -1.72773597e+00, -5.83161966e-01, -1.23828420e+00,
        7.92906253e+01, -2.04200635e+00, -3.80553587e-01,  8.20922389e-04,
        5.60347879e-01,  3.41695867e-01,  9.78048662e-01, -1.52109959e-01,
        6.57095859e-01]

### Aktueller Test

In [ ]:
# Test ob Q[:,:,:,0] ungleich Null und Messung des Einflusses der Parameter,
# Nelder-Mead, vorherige Datenbereinigung, 30k max iter zu besseren Ergebnissen führt
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter:")
print(likelihood)

# Code Graveyard

Vorherige Definition der Verteilungsfunktion von O_it über Schleifen

##### Definition der Erfolgswahrscheinlichkeit p_it als 1 x anzhal_states array 
def p_alt(iid, t, alpha_o, beta_o):
    LO_iddt = df_LO.loc[df_LO['iid'] == iid, t].values[0]
    p = exp(alpha_o + beta_o * np.log(LO_iddt)) / (1 + exp(alpha + beta * log(LO_iddt)))
    return p

#Wahrscheinlichkeiten werden in Array von Arrays zurückgegeben
def F_O_iidt_alt(alpha_o, beta_o):
    # P_O_iidt_speicher als Zwischenspeicher
    P_O_iidt_speicher_gesamt = []
    
    for iid in trans["iid"]:
        # P_O_iidt_speicher als Zwischenspeicher für jede iid
        P_O_iidt_speicher_jede_iid = []
        
        for t in range(1, 16):
            # Defintion der Erfolgswahrscheinlichkeit p_it mit alpha_o_gegeben_s und beta_o als Paramater
            # erfolgs_p = p(iid, t, alpha_o, beta_o)
            
            # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
            EM_iidt = df_HY.loc[df_HY['iid'] == iid, t].values[0]
            
            # o_iidt: Anzahl geöffneter Mails von Kunde iid in Monat t
            o_iidt = df_NO.loc[df_NO['iid'] == iid, t].values[0]
            
            # Wahrscheinlichkeiten der Binomialverteilung für jeden Zustand
            P_O_it = binom.cdf(o_iidt, EM_iidt, p(iid, t, alpha_o, beta_o))
        
            # Speichern der Wahrscheinlichkeiten im Zwischenspeicher
            P_O_iidt_speicher_jede_iid = P_O_iidt_speicher_jede_iid.extend(P_O_it)

        # In einer neuen Zeile für jede iid in dem Zwischenspeicher abspeichern
        P_O_iidt_speicher_gesamt = np.vstack([P_O_iidt_speicher_gesamt, P_O_iidt_speicher])
        
    return P_O_iidt_speicher_gesamt


##### Tests mit Diagonalmatrizen
            # Erstellen des Namens der Diagonalmatrix
            # name = f"diagonal_matrix_PO_{iid}_{t}"
        
            # Erstellen der Diagonalmatrix mit den Wahrscheinlichkeiten und dem Namen
            # diagonal_matrix = np.diag(probabilities)
            # diagonal_matrix.name = name

# Code wo veruscht wurde die arrays mit namen zu identifizieren

#### Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state_t, monat):
    
    state_t_minusEins = state_t_minusEins -1
    state_t = state_t -1
    str_monat = str(monat)
    
    # Definition der X_t_minusEins
    # verwendete Parameter
    EM_monat_minusEins = df_HY[str_monat]
    Y_monat_minusEins = df_trans_werte[str_monat]
    O_monat_minusEins = df_NO[str_monat]
    # Definition
    X_t_minusEins = [int(O_monat_minusEins > 0), int(Y_monat_minusEins > 0),\
                     EM_monat_minusEins, np.square(EM_monat_minusEins)]
    
    array_v = alpha[12*state_t_minusEins + 4*state + state] + gamma[12*state_t_minusEins + 4*state + state] * X_t_minusEins

    return array_v

##### Definition der Transitionmatrix
def probability_Q_state_monat(alpha, gamma):
    
    ### Erstellung leerer anzahl_states x anzahl_states arrays für jeden Kunden und jeden Monat
    Q = []
    for iid in df_trans_werte["iid"]:
        for monat in range(2,16):
                empty_array = np.zeros((anzahl_states, anzahl_states))
                Q.append(empty_array)
                
                #name = f"matrix_Q_{iid}_{monat}"
                #empty_array.name = name
                
    for monat in range(2,16):
        
        for state_t_minusEins in range(1, anzahl_states +1):
            
            # Erste Spalte besteht aus Nullen, da eine Trasition zurück in state 1 nicht möglich ist
            for state in range(2, anzahl_states +1):
                
                q_array_iid_monat_state = Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state_t, monat)
                #name = f"Q_{iid}_{monat}"

                # Überführen zu df und Hinzufügen einer iid Spalte zur Identifikation
                q_array_iid_monat_state = pd.DataFrame(q_array_iid_monat_state)
                q_array_iid_monat_state["iid"] = df_trans_werte["iid"]

                # Speicherung der Werte in den leeren arrays
                for index in q_array_iid_monat_state:
                    #name = f"matrix_Q_{iid}_{monat}"
                    #array_iid_monat = Q[name]
                    
                    #Neue Definition ohne die Verwedung von Namen
                    Q[index][state_t_minusEins-1, state-1] =\
                                    q_array_iid_monat_state[index]
                   
                    #array_iid_monat[state_t_minusEins-1, state-1] =\
                                    #q_array_iid_monat_state[q_array_iid_monat_state["iid"] == "iid"]
    
    return Q

#### alpha_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2 sind state dependent
#### beta_o, r sind state independent
def likelihood_function(alpha, gamma, alpha_o, beta_o, delta_0, delta_1,\
                        alpha_p, beta_p_1, beta_p_2, r, theta):
    
    ### Erstellung einer Liste H die alle H_it enthalten
    H = []
    
    ### Erstellung leerer 1xanzahl_states arrays für jeden Kunden und jeden Monat
    for iid in df_trans_werte["iid"]:
        for monat in range(1,16):
                empty_array = np.zeros((1, anzahl_states))
                H.append(empty_array)
                
                name = f"diagonal_matrix_H_{iid}_{monat}"
                empty_array.name = name
                
    for monat in range(1,16):
        for state in range(1, anzahl_states +1):
            
            # Berechnung von arrays für die Verteilungsfunktionen
            F_O_state_monat_berechnung = F_O_state_monat(alpha_o, beta_o, state, monat)
            
            F_O_state_monat_berechnung_minusEins = F_O_state_monat_minusEins(alpha_o, beta_o, state, monat)
            
            F_Y_state_monat_berechnung =\
                            F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r)
            
            F_Y_state_monat_berechnung_minusEins =\
                            F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r)
            
            # Berechnung der gemeinsamen Verteilungen
            H_state_monat_berechnung =\
                            Frank_Copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung, theta) -\
                            Frank_Copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung, theta) -\
                            Frank_Copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung_minusEins, theta) +\
                            Frank_Copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung_minusEins, theta)
            
            # Überführen zu df und Hinzufügen einer iid Spalte zur Identifikation
            H_state_monat_berechnung = pd.DataFrame(H_state_monat_berechnung)
            H_state_monat_berechnung["iid"] = df_trans_werte["iid"]
            
            # Speicherung der Werte in den leeren arrays
            for iid in H_state_monat_berechnung["iid"]:
                name = f"diagonal_matrix_H_{iid}_{monat}"
                H[name][state-1] = H_state_monat_berechnung[H_state_monat_berechnung["iid"] == "iid"]
                
    # Überführen der Arrays in H in Diagonalmatrizen
    for array in H:
        H[array] = np.diag(H[array])
        
    # Berechnung der Likelihood Function
    likelihood = 1
    
    # Startverteilung ist (1, 0,..., 0)
    startverteilung = np.zeros((1, anzahl_states))
    startverteilung[0, 0] = 1
    
    # Transitionmatrix Q
    Q = probability_Q_state_monat(alpha, gamma)
    
    Q_iid_prod = []
    for iid in df_trans_werte["iid"]:
        Q_iid = np.dot(startverteilung, H[f"diagonal_matrix_H_{iid}_1"])

        for monat in range(2,16):
            name = f"diagonal_matrix_H_{iid}_{monat}"
            name_Q = f"matrix_Q_{iid}_{monat}"
            H_iid_monat = np.dot(H[name], Q[name_Q])
            H_iid = np.dot(H_iid, H_iid_monat)
            
        Q_iid_prod.append(np.dot(Q_iid, H_iid))
        
    return np.prod(Q_iid_prod)

# ZINBD Verteilungsfunktion mit Model modelliert

#### Definition der Wahrscheinlichkeit des Eintretens eines Erfolges im Einzelversuch phi als anzahl_iids x T df
def phi_state_monat(delta_0, delta_1, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    LY_monat = df_LY[str_monat]
    array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))
    
    return array_phi

#### Definition der lambdaY als anzahl_iids x 1 array für monat und state
def lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    
    array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
    
    return array_lambdaY

#### Array der Verteilungsfunktion der Käufe Y für Monat und state
def F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # Erstellen Sie ein Zero-Inflated Negative Binomial Modell
    zinb_model = ZeroInflatedNegativeBinomialP(endog=None, exog=None, exposure=None, offset=None, missing='none')

    # Setzen Sie die offenen Parameter als Modellparameter
    zinb_model.update({'params': [1- phi_state_monat(delta_0, delta_1, state, monat),\
                                  lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat),\
                                  r]})
    
    # y_iid ist das DataFrame mit der Anzahl an Käufen
    y_monat = df_trans_werte[str_monat]
    
    # Verteilungsfunktion der ZINBD für jeden Zustand state
    F_Y_it = zinb_model.cdf(y_monat)
       
    return F_Y_it

#### Array der Verteilungsfunktion der Käufe Y-1 für Monat und state
def F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r):
    
    # Berechnung der Werte minus Eins für spätere Berechnung der gemeinsamen Verteilung via Frank-Copula
    df_trans_werte_minusEins = df_trans_werte.loc[:, df_NO_werte.columns != 'iid'] -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # Erstellen Sie ein Zero-Inflated Negative Binomial Modell
    zinb_model = ZeroInflatedNegativeBinomialP(endog=None, exog=None, exposure=None, offset=None, missing='none')

    # Setzen Sie die offenen Parameter als Modellparameter
    zinb_model.update({'params': [1- phi_state_monat(delta_0, delta_1, state, monat),\
                                  lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat),\
                                  r]})
    
    # y_iid ist das DataFrame mit der Anzahl an Käufen
    y_monat = df_trans_werte_minusEins[str_monat]
    
    # Verteilungsfunktion der ZINBD für jeden Zustand state
    F_Y_it = zinb_model.cdf(y_monat)
       
    return F_Y_it

# CEOM als BD

#### Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def probability_O_state_monat(alpha_o, beta_o, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # Erhalten der Werte für die Verarbeitung in der Formel
    LO_monat = df_LO[str_monat]
    array_p = np.exp(alpha_o[state] + beta_o[0] * np.log(LO_monat)) / (1 + np.exp(alpha_o[state] + beta_o[0] * np.log(LO_monat)))

    return array_p

#### Array der Verteilungsfunktion der Öffungen O für Monat und state
def F_O_state_monat(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_werte[str_monat]
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    
    probs = probability_O_state_monat(alpha_o, beta_o, state, monat)
    array_F_O_berechnung = binom.cdf(o_monat, EM_monat, probs)
         
    return array_F_O_berechnung

#### Array der Verteilungsfunktion der Öffungen O -1 für Monat und state
def F_O_state_monat_minusEins(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_minusEins[str_monat]
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    
    probs = probability_O_state_monat(alpha_o, beta_o, state, monat)
    array_F_O_berechnung = binom.cdf(o_monat, EM_monat, probs)

    return array_F_O_berechnung

#### Umständliche Defiitio Matrixmultiplikation
    Q_iid_prod = np.array([1])
    for j in range(len(df_trans_werte)):
        Q_iid = np.dot(startverteilung, H[15 * j])
        H_iid = np.eye(anzahl_states)
        for monat in range(2,16):
            H_iid_monat = np.dot(H[j * 15 + (monat-1)], Q[j * 14 + (monat-2)])
            H_iid = np.dot(H_iid, H_iid_monat)
            
        Q_iid_prod.append(np.dot(Q_iid, H_iid))